In [ ]:
# =========================
# BLOCK A — PREPROCESSING (GENE-SUMMED DOSAGES; MATCHES YOUR BASELINE)
# =========================
import os
import numpy as np
import pandas as pd
from collections import defaultdict
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer

print("--- Starting Data Preprocessing (gene-summed SNP dosages) ---")

# --- File paths (your originals) ---
main_data_path    = '/Users/authorname/Downloads/all_chr_merged.parquet'
gwas_catalog_path = '/Users/authorname/Downloads/MONDO_0004975_associations_export.tsv'
string_links_path = '/Users/authorname/Downloads/9606.protein.links.v12.0.txt'      # unused here
string_info_path  = '/Users/authorname/Downloads/9606.protein.info.v12.0.txt'       # unused here

out_dir = "/Users/authorname/Desktop/Projects/Genomics_LGBM(Genomics+Meta)_RATIOS_fixed_rfe"
os.makedirs(out_dir, exist_ok=True)

# --- Load SNP matrix + GWAS mapping (your logic) ---
df = pd.read_parquet(main_data_path)
gwas_df = pd.read_csv(gwas_catalog_path, sep='\t')

gwas_df['riskAllele_cleaned'] = gwas_df['riskAllele'].astype(str).str.split('-').str[0].str.strip()
gwas_df['first_gene'] = gwas_df['mappedGenes'].astype(str).str.split(',').str[0].str.strip()
rs_map = gwas_df.dropna(subset=['riskAllele_cleaned', 'first_gene']) \
                .set_index('riskAllele_cleaned')['first_gene'].to_dict()

df.columns = [c.strip() for c in df.columns]
snp_cols = [c for c in df.columns if c.startswith('rs') or c.startswith('chr')]
all_mapped_snps = {s: rs_map.get(s.split('_')[0]) for s in snp_cols if rs_map.get(s.split('_')[0])}

# --- SNP QC (same thresholds as baseline) ---
print("\n--- SNP QC ---")
snps_to_map = list(all_mapped_snps.keys())
maf = df[snps_to_map].apply(
    lambda x: (2*x.eq(2).sum() + x.eq(1).sum()) / (2*x.notna().sum()),
    axis=0
)
snps_to_remove_maf = maf[maf < 0.01].index.tolist()

missing_rates = df[snps_to_map].isna().mean()
snps_to_remove_callrate = missing_rates[missing_rates > 0.05].index.tolist()

all_snps_to_remove = set(snps_to_remove_maf + snps_to_remove_callrate)
all_mapped_snps_qc = {snp: gene for snp, gene in all_mapped_snps.items() if snp not in all_snps_to_remove}

df_qc = df.drop(columns=list(all_snps_to_remove), errors="ignore")
snp_cols_qc = list(all_mapped_snps_qc.keys())
print(f"Retained {len(snp_cols_qc)} SNPs after QC.")

# --- Label assignment (your definitions) ---
print("\n--- Label assignment ---")
df_qc['age_death'] = pd.to_numeric(df_qc['age_death'].astype(str).replace('90+', '90', regex=False), errors='coerce')

conditions = [
    (df_qc['ceradsc'].isin([1, 2])) & (df_qc['cogdx'].isin([4, 5])),
    (df_qc['ceradsc'].isin([1, 2])) & ~(df_qc['cogdx'].isin([4, 5])),
    ~(df_qc['ceradsc'].isin([1, 2])) & (df_qc['cogdx'].isin([4, 5])),
    ~(df_qc['ceradsc'].isin([1, 2])) & ~(df_qc['cogdx'].isin([4, 5]))
]
choices = ['Typical AD', 'Resilient', 'Symptomatic Non-AD', 'Healthy Control']
df_qc['patient_group'] = np.select(conditions, choices, default='Unknown')
df_qc = df_qc[df_qc['patient_group'] != 'Unknown'].reset_index(drop=True)

print("Patient Group Counts:")
print(df_qc['patient_group'].value_counts())

# --- Demographics + SNP handling (mirror baseline) ---
numeric_features     = ['age_death', 'educ']
categorical_features = ['apoe_genotype']
binary_features      = ['msex']
snp_features         = snp_cols_qc

feature_df = df_qc[['projid'] + numeric_features + categorical_features + binary_features + snp_features].copy()

# Impute numeric only
imputer_num = SimpleImputer(strategy='median')
feature_df[numeric_features] = imputer_num.fit_transform(feature_df[numeric_features])

# Keep SNP NaNs; round observed to {0,1,2}
snp_block = feature_df[snp_features].astype(float)
mask = ~snp_block.isna()
rounded = np.empty_like(snp_block.values)
rounded[:] = np.nan
rounded[mask.values] = np.rint(snp_block.values[mask.values])
snp_block = pd.DataFrame(rounded, columns=snp_features, index=feature_df.index)
feature_df[snp_features] = snp_block

feature_df[binary_features] = feature_df[binary_features].astype(int)

# Sum SNPs → genes (ignore NaN; keep NaN if all missing)
print("\n--- Building gene-summed dosage matrix ---")
gene_to_snps = defaultdict(list)
for snp, gene in all_mapped_snps_qc.items():
    gene_to_snps[gene].append(snp)

gene_dosage_df = pd.DataFrame(index=feature_df.index)
for gene, snp_list in gene_to_snps.items():
    vals = feature_df[snp_list].to_numpy(dtype=float)
    gsum = np.nansum(vals, axis=1)
    all_missing = np.isnan(vals).all(axis=1)
    gsum[all_missing] = np.nan
    gene_dosage_df[gene] = gsum

# APOE OHE
try:
    ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
except TypeError:
    ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
apoe_encoded = ohe.fit_transform(feature_df[['apoe_genotype']])
apoe_df = pd.DataFrame(apoe_encoded, columns=ohe.get_feature_names_out(), index=feature_df.index)

# Final feature table: demographics + APOE OHE + gene_sums
X_gene_based = pd.concat(
    [feature_df[numeric_features + binary_features].reset_index(drop=True),
     apoe_df.reset_index(drop=True),
     gene_dosage_df.reset_index(drop=True)],
    axis=1
)
X_gene_based.columns = X_gene_based.columns.astype(str)

# Labels & encoders (to match your baseline later)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df_qc['patient_group'].astype(str).values)
y_labels = df_qc['patient_group'].astype(str).values

# Stratifier exactly like your baseline
y_strat = (df_qc['patient_group'].astype(str) + "_" + df_qc['msex'].astype(int).astype(str)).values

# Keep list of pure gene columns (for ratio generation later)
non_gene_cols = set(numeric_features + binary_features + list(apoe_df.columns))
gene_cols = [c for c in X_gene_based.columns if c not in non_gene_cols]

# Groups if needed elsewhere
groups = df_qc['projid'].astype(str).values

print(f"\nX_gene_based shape: {X_gene_based.shape}")
print("Label to Class Name mapping:")
for i, class_name in enumerate(label_encoder.classes_):
    print(f"{i}: {class_name}")
print(f"Total genes available for ratio generation: {len(gene_cols)}")

In [ ]:
# =========================
# BLOCK B — READ BASELINE GENOMICS PICKLES & COLLECT STABLE GENES
# =========================
import os, pickle
from collections import Counter, defaultdict

print("\n--- Reading baseline GENOMICS LGBM pickles for stable features (≥2/5 seeds) ---")

baseline_models_dir = "/Users/authorname/Desktop/Projects/ml4h_project/LGBM(Genes+Demo)"

# Match EXACT names you saved with in baseline training
classes = [
    "Healthy_Control",
    "Resilient",
    "Symptomatic_Non-AD",   # note the dash
    "Typical_AD"
]
seeds = [1, 2, 3, 4, 5]

def _get_trained_estimator_and_features(automl_obj):
    model = getattr(automl_obj, "model", None)
    est = getattr(model, "estimator", model)
    if est is None:
        return None, None
    if hasattr(automl_obj, "feature_names_in_") and automl_obj.feature_names_in_ is not None:
        feat_names = list(automl_obj.feature_names_in_)
    elif hasattr(est, "feature_name_") and est.feature_name_ is not None:
        feat_names = list(est.feature_name_)
    else:
        feat_names = None
    return est, feat_names

feature_counts_per_class = defaultdict(Counter)

for seed in seeds:
    for cls in classes:
        pkl_path = os.path.join(baseline_models_dir, f"seed{seed}_{cls}_automl.pkl")
        if not os.path.exists(pkl_path):
            print(f"[WARN] Missing: {pkl_path}")
            continue
        try:
            with open(pkl_path, "rb") as f:
                automl = pickle.load(f)
            est, feature_names = _get_trained_estimator_and_features(automl)
            if est is None or not hasattr(est, "feature_importances_"):
                print(f"[WARN] No feature importances in {pkl_path}")
                continue
            importances = est.feature_importances_
            nonzero_genes = [f for f, imp in zip(feature_names, importances) if imp > 0 and f in gene_cols]
            feature_counts_per_class[cls].update(nonzero_genes)
        except Exception as e:
            print(f"[ERROR] {pkl_path}: {e}")

# keep genes seen in ≥2 seeds
shared_features_per_class = {
    cls: [feat for feat, cnt in cnts.items() if cnt >= 2]
    for cls, cnts in feature_counts_per_class.items()
}

for cls in classes:
    feats = shared_features_per_class.get(cls, [])
    print(f"[{cls}] Stable genes (≥2 seeds): {len(feats)}")

In [ ]:
# =========================
# BLOCK C — RATIO TRAINING WITH RFE (GENE-BASED) - CORRECTED
# =========================

import os
out_dir = "/Users/authorname/Desktop/Projects/Genomics_LGBM(Genomics+Meta)_RATIOS_fixed_rfe"
os.makedirs(out_dir, exist_ok=True)


import os
import numpy as np
import pandas as pd
from itertools import combinations
from flaml import AutoML
from sklearn.metrics import roc_auc_score
from sklearn.feature_selection import RFE
from lightgbm import LGBMClassifier

# ---- Config (assumes variables from Blocks A & B are loaded) ----
# `out_dir`, `label_encoder`, `y_labels`, `y_strat`, `groups`,
# `X_gene_based`, `gene_cols`, and `shared_features_per_class`
# should all be available from the previous blocks.

# --- 💡 FIX: Use the class names with underscores to match the keys ---
# --- in the `shared_features_per_class` dictionary from Block B. ---
classes = [
    "Healthy_Control",
    "Resilient",
    "Symptomatic_Non-AD",
    "Typical_AD"
]
seeds = [1, 2, 3, 4, 5]

def safe_cls(c):
    return c.replace(" ", "_").replace("/", "-")

# ---- Gene-only matrix for creating ratios ----
X_genes_only = X_gene_based[gene_cols].copy()

# ---- Custom group-aware stratified split function (no changes needed) ----
def group_stratified_shuffle_split(df_index, strata_all, groups_all, test_size=0.30, random_state=0):
    rng = np.random.RandomState(random_state)
    data = pd.DataFrame({"idx": df_index, "strata": strata_all, "group": groups_all})
    grp_mode = data.groupby("group")["strata"].agg(lambda s: s.value_counts().idxmax())
    grp_mode = grp_mode.sample(frac=1.0, random_state=random_state)

    train_groups, test_groups = [], []
    for _, grp_ids in grp_mode.groupby(grp_mode.values):
        g_list = list(grp_ids.index)
        rng.shuffle(g_list)
        n_test = max(1, int(round(test_size * len(g_list))))
        test_groups.extend(g_list[:n_test])
        train_groups.extend(g_list[n_test:])

    train_mask = np.isin(groups_all, train_groups)
    test_mask  = np.isin(groups_all, test_groups)
    return np.where(train_mask)[0], np.where(test_mask)[0]

# ---- Main Training Loop ----
for cls in classes:
    # 1. Select stable genes for the current class
    # Now, `cls` will be "Healthy_Control", which correctly finds the 7 genes.
    class_genes = shared_features_per_class.get(cls, [])
    print(f"\n[{cls}] Using {len(class_genes)} stable genes for ratio generation...")

    if len(class_genes) < 2:
        print(f"-> Skipping [{cls}]: Not enough genes to form ratios (need at least 2).")
        continue

    # 2. Generate all pairwise ratios from the selected genes
    ratio_cols = []
    ratio_data = []
    epsilon = 1e-8  # Small value to prevent division by zero

    for g1, g2 in combinations(class_genes, 2):
        rname = f"{g1}_div_{g2}"
        ratio_cols.append(rname)
        # Add epsilon for numerical stability
        ratio_data.append(X_genes_only[g1].values / (X_genes_only[g2].values + epsilon))

    X_ratio_cls = pd.DataFrame(np.column_stack(ratio_data), columns=ratio_cols, index=X_genes_only.index)
    print(f"-> Generated {X_ratio_cls.shape[1]} ratio features for [{cls}].")

    # 3. Loop through seeds to perform train/test split, RFE, and model training
    for seed in seeds:
        # A. Split data into training and testing sets
        tr_idx, te_idx = group_stratified_shuffle_split(
            df_index=np.arange(len(X_ratio_cls)),
            strata_all=y_strat,
            groups_all=groups,
            test_size=0.30,
            random_state=seed,
        )

        # B. Create initial data splits with all generated ratios
        X_train_full = X_ratio_cls.iloc[tr_idx]
        X_test_full  = X_ratio_cls.iloc[te_idx]
        y_train_full = pd.Series(y_labels).iloc[tr_idx].values
        y_test_full  = pd.Series(y_labels).iloc[te_idx].values

        print(f"[{cls} | Seed {seed}] Train/Test split: {len(tr_idx)} / {len(te_idx)}")

        # C. Create binary labels for one-vs-all classification
        # We must convert the underscore version back to the space version to match `y_labels`
        y_train = (y_train_full == cls.replace("_", " ")).astype(int)
        y_test  = (y_test_full == cls.replace("_", " ")).astype(int)


        # D. Perform RFE on the TRAINING data ONLY
        rfe_model = LGBMClassifier(n_estimators=100, random_state=seed)
        n_select  = min(100, X_train_full.shape[1]) # Select up to 100 features
        rfe = RFE(estimator=rfe_model, n_features_to_select=n_select, step=0.1)
        rfe.fit(X_train_full, y_train)

        # E. Filter both train and test sets using the features selected by RFE
        selected_cols = list(X_train_full.columns[rfe.get_support()])
        X_train = X_train_full[selected_cols]
        X_test  = X_test_full[selected_cols]
        print(f"-> Post-RFE features selected: {len(selected_cols)}")

        # F. Train the final model using AutoML
        automl = AutoML()
        settings = {
            "time_budget": 100,
            "metric": "roc_auc",
            "task": "classification",
            "eval_method": "cv",
            "estimator_list": ["lgbm"],
            "log_file_name": os.path.join(out_dir, f"flaml_seed{seed}_{safe_cls(cls)}.log"),
            "seed": seed,
        }

        automl.fit(X_train=X_train, y_train=y_train, **settings)
        automl.pickle(os.path.join(out_dir, f"seed{seed}_{safe_cls(cls)}_automl.pkl"))

        # G. Evaluate performance and save results
        y_score = automl.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(y_test, y_score) if (y_test.sum() > 0 and y_test.sum() < len(y_test)) else float("nan")
        print(f"  -> FINAL AUC = {auc:.4f}\n")

        out_csv = os.path.join(out_dir, f"seed{seed}_{safe_cls(cls)}.csv")
        pd.DataFrame({
            "y_true": y_test.astype(int),
            "y_score": y_score.astype(float)
        }).to_csv(out_csv, index=False)